<a href="https://colab.research.google.com/github/hasiburrahman1/NLP/blob/master/complex_type_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [238]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [239]:
import numpy as np 
import pandas as pd 
import os
import re
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, SimpleRNN, GRU, LSTM, Bidirectional, Dropout, Input, Conv2D, MaxPool2D
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.optimizers import Adam
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten, Input, GlobalMaxPooling1D

In [240]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import tensorflow as tf
import collections
import unicodedata
import re
import numpy as np
import os
import io
import time
import xlrd
import csv

In [241]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [242]:
'''
cols_to_use = ['Question', 'Answer'] # or [0,1,2,3]
df = pd.read_csv('data.csv', usecols= cols_to_use)
'''

"\ncols_to_use = ['Question', 'Answer'] # or [0,1,2,3]\ndf = pd.read_csv('data.csv', usecols= cols_to_use)\n"

In [243]:
'''
mport pandas
df = pandas.read_excel(r"/test.xlsx")
df = df.drop('label', 1)
#print the column names
print(df.columns)

#get the values for a given column
#values = df['collumn_name'].values

#get a data frame with selected columns
FORMAT = ['Question', 'Answer']
df_selected = df[FORMAT]


df.to_csv('dataset_csv.csv')
!cp dataset_csv.csv "drive/My Drive/"

df.describe()
'''

'\nmport pandas\ndf = pandas.read_excel(r"/test.xlsx")\ndf = df.drop(\'label\', 1)\n#print the column names\nprint(df.columns)\n\n#get the values for a given column\n#values = df[\'collumn_name\'].values\n\n#get a data frame with selected columns\nFORMAT = [\'Question\', \'Answer\']\ndf_selected = df[FORMAT]\n\n\ndf.to_csv(\'dataset_csv.csv\')\n!cp dataset_csv.csv "drive/My Drive/"\n\ndf.describe()\n'

---------------------------**Start**-------------------------------------------------------------------------------------------------------------------------------------

In [244]:
# dataset path
#path_to_file = "/datasetCleaned.txt"
path_to_file = "/content/drive/My Drive/Colab Notebooks/dataset/test_Text_Document.txt"

In [245]:
def preprocess_sentence(w):
#     remove punctuations from lines
    w = re.sub(r"([?.!,¿।])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    w = w.rstrip().strip()
    #print(w)
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    #print(w)
    return w

In [246]:
def vocabularyCounter(dataQuestion, dataAnswer):
    # setup Vocabulary 
    question_words = collections.Counter([word for sentence in dataQuestion for word in sentence.split()])
    answer_words = collections.Counter([word for sentence in dataAnswer for word in sentence.split()])
    
    print('Total English words: {}'.format(len([word for sentence in dataQuestion for word in sentence.split()])))
    print('Unique English words: {}'.format(len(question_words)))
    print('Most Common Words: "' + '" "'.join(list(zip(*question_words.most_common(10)))[0]) + '"\n')

    print('Total Bangla words: {}'.format(len([word for sentence in dataAnswer for word in sentence.split()])))
    print('Unique Bangla words: {}'.format(len(answer_words)))
    print('Most Common Words: "' + '" "'.join(list(zip(*answer_words.most_common(10)))[0]) + '"')

In [247]:

# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, BANGLA]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    
    """
    test test test
    """
    #print(lines)
    #print("\n")
    #print(num_examples)
    """
    test test test
    """

    question = []
    answer = []
    for l in lines[:num_examples]:
        #print(l)
        w = l.split('\t')
        """
        test test test
        """
        #print(w)
        #print("\n")
        """
        test test test
        """
        question.append(w[0])
        answer.append(w[1])
    vocabularyCounter(question, answer) 
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    #print(word_pairs)
#     rearrange dataset src and target
    for indd in range(len(word_pairs)):
        temp = word_pairs[indd][0]
        word_pairs[indd][0] = word_pairs[indd][1]
        word_pairs[indd][1] = temp
    #print(word_pairs)
    return zip(*word_pairs)


In [248]:
# preprocessing function declear and vocabulary checking
bn, en = create_dataset(path_to_file, None)

#print("\n\n", en[4])
#print(bn)

Total English words: 40
Unique English words: 9
Most Common Words: "Do" "lincRNAs" "play" "a" "role" "human" "cancer?" "1in" "in"

Total Bangla words: 66
Unique Bangla words: 50
Most Common Words: "lincRNAs" "of" "and" "to" "polyadenylation" "in" "cancer" "we" "the" "Genome-wide"


In [249]:
#function declearation for 
#     1. length of sentence
#https://stackoverflow.com/questions/51956000/what-does-keras-tokenizer-method-exactly-do
#https://stackoverflow.com/questions/61661160/comparison-of-tf-keras-preprocessing-text-tokenizer-and-tfds-features-text-tok
def max_length(tensor):
    return max(len(t) for t in tensor)

#     2. tokenizer
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer
#     3. load dataset
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang  = create_dataset(path, num_examples)
    
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer
'''
   Total English words: 40
Unique English words: 9
Most Common Words: "Do" "lincRNAs" "play" "a" "role" "human" "cancer?" "1in" "in"

Total Bangla words: 66
Unique Bangla words: 50
Most Common Words: "lincRNAs" "of" "and" "to" "polyadenylation" "in" "cancer" "we" "the" "Genome-wide"
(array([[ 1,  2,  3,  4,  5,  6, 12,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],
        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10]], dtype=int32),
 array([[ 1, 13, 14,  7, 15, 16,  5,  2,  8,  9, 10, 17,  3,  4,  0,  0,
          0,  0,  0,  0],
        [ 1, 18, 19, 11, 20,  2,  6, 21, 22,  9, 23, 24,  5, 25, 26,  3,
          4,  0,  0,  0],
        [ 1, 27, 28,  5,  2, 29, 30, 31, 32, 33, 34,  3,  4,  0,  0,  0,
          0,  0,  0,  0],
        [ 1,  6, 35, 12, 36,  5,  2,  8, 37,  6, 10, 38, 39, 40, 41,  3,
          4,  0,  0,  0],
        [ 1, 42, 11, 43, 44, 45, 46,  7, 47, 48,  6, 49, 12, 50, 51,  2,
          7, 52,  3,  4]], dtype=int32),
 <keras_preprocessing.text.Tokenizer at 0x7f9045bf34a8>,
 <keras_preprocessing.text.Tokenizer at 0x7f9045d264a8>)
 ''' 

'\n   Total English words: 40\nUnique English words: 9\nMost Common Words: "Do" "lincRNAs" "play" "a" "role" "human" "cancer?" "1in" "in"\n\nTotal Bangla words: 66\nUnique Bangla words: 50\nMost Common Words: "lincRNAs" "of" "and" "to" "polyadenylation" "in" "cancer" "we" "the" "Genome-wide"\n(array([[ 1,  2,  3,  4,  5,  6, 12,  7,  8,  9, 10],\n        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],\n        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],\n        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10],\n        [ 1,  2,  3,  4,  5,  6, 11,  7,  8,  9, 10]], dtype=int32),\n array([[ 1, 13, 14,  7, 15, 16,  5,  2,  8,  9, 10, 17,  3,  4,  0,  0,\n          0,  0,  0,  0],\n        [ 1, 18, 19, 11, 20,  2,  6, 21, 22,  9, 23, 24,  5, 25, 26,  3,\n          4,  0,  0,  0],\n        [ 1, 27, 28,  5,  2, 29, 30, 31, 32, 33, 34,  3,  4,  0,  0,  0,\n          0,  0,  0,  0],\n        [ 1,  6, 35, 12, 36,  5,  2,  8, 37,  6, 10, 38, 39, 40, 41,  3,\n          4,  0,  0,  0],\n        [ 1,

In [250]:
# Try experimenting with the size of that dataset
num_examples = 500
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)


print(input_tensor)
print("\n")
print(inp_lang)


# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print("Size:- Train_X-", len(input_tensor_train), "Train_Y-", len(target_tensor_train), "Test_X-", len(input_tensor_val), "Test_Y-", len(target_tensor_val))

Total English words: 40
Unique English words: 9
Most Common Words: "Do" "lincRNAs" "play" "a" "role" "human" "cancer?" "1in" "in"

Total Bangla words: 66
Unique Bangla words: 50
Most Common Words: "lincRNAs" "of" "and" "to" "polyadenylation" "in" "cancer" "we" "the" "Genome-wide"
[[ 1  2  3  4  5  6 12  7  8  9 10]
 [ 1  2  3  4  5  6 11  7  8  9 10]
 [ 1  2  3  4  5  6 11  7  8  9 10]
 [ 1  2  3  4  5  6 11  7  8  9 10]
 [ 1  2  3  4  5  6 11  7  8  9 10]]


Size:- Train_X- 4 Train_Y- 4 Test_X- 1 Test_Y- 1


In [251]:
# showing example of tokenization
def convert(lang, tensor):
  for t in tensor:
    print("t t t",t)

    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))
    
    
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
t t t 1
1 ----> <start>
t t t 2
2 ----> do
t t t 3
3 ----> lincrnas
t t t 4
4 ----> play
t t t 5
5 ----> a
t t t 6
6 ----> role
t t t 11
11 ----> 1in
t t t 7
7 ----> human
t t t 8
8 ----> cancer
t t t 9
9 ----> ?
t t t 10
10 ----> <end>

Target Language; index to word mapping
t t t 1
1 ----> <start>
t t t 18
18 ----> as
t t t 19
19 ----> expected
t t t 11
11 ----> we
t t t 20
20 ----> found
t t t 2
2 ----> lincrnas
t t t 6
6 ----> to
t t t 21
21 ----> be
t t t 22
22 ----> present
t t t 9
9 ----> in
t t t 23
23 ----> cd14
t t t 24
24 ----> monocytes
t t t 5
5 ----> of
t t t 25
25 ----> ra
t t t 26
26 ----> patients
t t t 3
3 ----> .
t t t 4
4 ----> <end>
t t t 0
t t t 0
t t t 0
